In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

diabetes_dataframe = pd.read_csv(
    "https://raw.githubusercontent.com/plotly/datasets/master/diabetes.csv"
)

# Shuffle the data points
diabetes_dataframe = shuffle(diabetes_dataframe)

diabetes_dataframe_label = diabetes_dataframe.pop("Outcome")
diabetes_dataframe_features = diabetes_dataframe

# Split into training and testing sets
(
    diabetes_dataframe_training_features,
    diabetes_dataframe_test_features,
    diabetes_dataframe_training_label,
    diabetes_dataframe_test_label,
) = train_test_split(
    diabetes_dataframe_features, diabetes_dataframe_label, test_size=0.2
)

print(len(diabetes_dataframe))
print(len(diabetes_dataframe_training_features))
print(len(diabetes_dataframe_training_label))
print(len(diabetes_dataframe_test_features))
print(len(diabetes_dataframe_test_label))

768
614
614
154
154


In [5]:
from sklearn.preprocessing import StandardScaler

diabetes_dataframe_training_features_normalized = pd.DataFrame(
    StandardScaler().fit_transform(diabetes_dataframe_training_features),
    columns = diabetes_dataframe_training_features.columns
)

diabetes_dataframe_test_features_normalized = pd.DataFrame(
    StandardScaler().fit_transform(diabetes_dataframe_test_features),
    columns = diabetes_dataframe_test_features.columns
)

print(diabetes_dataframe_training_features.head())
print(diabetes_dataframe_training_features_normalized.head())

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
576            6      108             44             20      130  24.0   
637            2       94             76             18       66  31.6   
22             7      196             90              0        0  39.8   
646            1      167             74             17      144  23.4   
597            1       89             24             19       25  27.8   

     DiabetesPedigreeFunction  Age  
576                     0.813   35  
637                     0.649   23  
22                      0.451   41  
646                     0.447   33  
597                     0.559   21  
   Pregnancies   Glucose  BloodPressure  SkinThickness   Insulin       BMI  \
0     0.674409 -0.409944      -1.286020      -0.005579  0.438926 -0.989742   
1    -0.521511 -0.869919       0.368818      -0.132457 -0.104131 -0.018319   
2     0.973389  2.481333       1.092810      -1.274359 -0.664158  1.029795   
3    -0.820490  1.528

In [6]:
from sklearn.linear_model import LogisticRegression

model2 = LogisticRegression()
model2.fit(
    diabetes_dataframe_training_features_normalized, diabetes_dataframe_training_label
)

prob = model2.predict_proba(diabetes_dataframe_test_features_normalized)

diabetes_dataframe_test_results2 = diabetes_dataframe_test_features.copy()

diabetes_dataframe_test_results2["Outcome"] = diabetes_dataframe_test_label
diabetes_dataframe_test_results2["Predicted Outcome"] = prob[:, 1]

print(diabetes_dataframe_test_results2.head())

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
71             5      139             64             35      140  28.6   
237            0      179             90             27        0  44.1   
192            7      159             66              0        0  30.4   
145            0      102             75             23        0   0.0   
466            0       74             52             10       36  27.8   

     DiabetesPedigreeFunction  Age  Outcome  Predicted Outcome  
71                      0.411   26        0           0.304869  
237                     0.686   23        1           0.711093  
192                     0.383   36        1           0.582856  
145                     0.572   21        0           0.013124  
466                     0.269   22        0           0.036627  
